In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import missingno as msno
import plotly.express as px 
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import datetime

# Fehleranalyse in der Datei user.csv

Zu überprüfende Fehlertypen:
- Unmögliche Daten in Spalten (unrealistische user_age, account_created_date...)
- Falsche Datumsreihenfolge: first_active_date ≤ account_created_date ≤ first_booking_date
- Wenn first_booking_date = NaN, dann sollte destination_country NDF sein
- Tippfehler in Textspalten

In [21]:
# Daten laden
df_user = pd.read_csv('data/user.csv')

print(f"Anzahl der Zeilen: {len(df_user)}")
print(f"Anzahl der Spalten: {len(df_user.columns)}")
print(f"\nSpalten: {list(df_user.columns)}")

Anzahl der Zeilen: 213451
Anzahl der Spalten: 16

Spalten: ['user_id', 'account_created_date', 'first_active_timestamp', 'first_booking_date', 'user_gender', 'user_age', 'signup_platform', 'signup_process', 'user_language', 'marketing_channel', 'marketing_provider', 'first_tracked_affiliate', 'signup_application', 'first_device', 'first_web_browser', 'destination_country']


In [18]:
df_user.head()

,user_id,account_created_date,first_active_timestamp,first_booking_date,user_gender,user_age,signup_platform,signup_process,user_language,marketing_channel,marketing_provider,first_tracked_affiliate,signup_application,first_device,first_web_browser,destination_country
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,affiliate,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,FEMALE,38.0,affiliate,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,MALE,56.0,web,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,MALE,42.0,affiliate,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,web,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [ ]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213451 entries, 0 to 213450
Data columns (total 16 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   user_id                  213451 non-null  object 
 1   account_created_date     213451 non-null  object 
 2   first_active_timestamp   213451 non-null  int64  
 3   first_booking_date       88908 non-null   object 
 4   user_gender              213451 non-null  object 
 5   user_age                 125461 non-null  float64
 6   signup_platform          213451 non-null  object 
 7   signup_process           213451 non-null  int64  
 8   user_language            213451 non-null  object 
 9   marketing_channel        213451 non-null  object 
 10  marketing_provider       213451 non-null  object 
 11  first_tracked_affiliate  207386 non-null  object 
 12  signup_application       213451 non-null  object 
 13  first_device             213451 non-null  object 
 14  firs

In [22]:
# Datumsumwandlung
# first_active_date aus first_active_timestamp erstellen (nur Datum für Vergleiche)
df_user['first_active_timestamp'] = pd.to_datetime(df_user['first_active_timestamp'], format='%Y%m%d%H%M%S', errors='coerce')
df_user['first_active_date'] = df_user['first_active_timestamp'].dt.date

df_user['account_created_date'] = pd.to_datetime(df_user['account_created_date'], format='%Y-%m-%d', errors='coerce')
df_user['first_booking_date'] = pd.to_datetime(df_user['first_booking_date'], format='%Y-%m-%d', errors='coerce')

print("Datumsbereiche:")
print(f"  first_active_date: {df_user['first_active_timestamp'].min()} - {df_user['first_active_timestamp'].max()}")
print(f"  account_created_date: {df_user['account_created_date'].min()} - {df_user['account_created_date'].max()}")
print(f"  first_booking_date: {df_user['first_booking_date'].min()} - {df_user['first_booking_date'].max()}")

Datumsbereiche:
  first_active_date: 2009-03-19 04:32:55 - 2014-06-30 23:58:24
  account_created_date: 2010-01-01 00:00:00 - 2014-06-30 00:00:00
  first_booking_date: 2010-01-02 00:00:00 - 2015-06-29 00:00:00


In [24]:
df_user['account_created_date'][0]

Timestamp('2010-06-28 00:00:00')

## 1. Altersanalyse

In [32]:
# Altersstatistik
print("Altersstatistik:")
print(df_user['user_age'].describe())

# Unrealistische Werte prüfen
young_ages = df_user[df_user['user_age'] < 18]
old_ages = df_user[df_user['user_age'] > 90]
age_over_80 = df_user[(df_user['user_age'] > 80) & (df_user['user_age'] <= 90)]
year_ages = df_user[df_user['user_age'] >= 2000]

print(f"\n❌ ALTERSFEHLER:")
print(f"  • Alter < 18 Jahre: {len(young_ages)} Einträge")
print(f"  • Alter > 90 Jahre: {len(old_ages)} Einträge")
print(f"  • Alter 81-90 Jahre: {len(age_over_80)} Einträge")
print(f"  • Alter >= 2000 (Geburtsjahr statt Alter): {len(year_ages)} Einträge")

print(f"\nBeispiele für unrealistisches Alter:")
display(df_user[df_user['user_age'] >= 2000][['user_id', 'user_age', 'account_created_date']].head(10))

Altersstatistik:
count    125461.000000
mean         49.668335
std         155.666612
min           1.000000
25%          28.000000
50%          34.000000
75%          43.000000
max        2014.000000
Name: user_age, dtype: float64

❌ ALTERSFEHLER:
  • Alter < 18 Jahre: 158 Einträge
  • Alter > 90 Jahre: 2543 Einträge
  • Alter 81-90 Jahre: 228 Einträge
  • Alter >= 2000 (Geburtsjahr statt Alter): 750 Einträge

Beispiele für unrealistisches Alter:


,user_id,user_age,account_created_date
388,v2x0ms9c62,2014.0,2010-04-11
673,umf1wdk9uc,2014.0,2010-05-25
1040,m82epwn7i8,2014.0,2010-07-14
1177,2th813zdx7,2013.0,2010-07-25
1200,3amf04n3o3,2014.0,2010-07-27
1239,6vpmryt377,2014.0,2010-07-30
1257,uxy91xb5p2,2014.0,2010-08-01
1308,bno0vva4uz,2014.0,2010-08-06
1474,h3rrmak4tu,2014.0,2010-08-21
1502,fou0j7fhnm,2014.0,2010-08-24


## 2. Datumsreihenfolge

In [ ]:
# account_created_date > first_booking_date
num_errors = ((df_user['first_booking_date'].notna()) & 
              (df_user['account_created_date'] > df_user['first_booking_date'])).sum()

print(f"❌ account_created_date > first_booking_date: {num_errors} Einträge")
print(f"   (Konto NACH erster Buchung erstellt - unmöglich)")

if num_errors > 0:
    print(f"\nBeispiele:")
    display(df_user[(df_user['account_created_date'] > df_user['first_booking_date'])][['user_id', 'account_created_date', 'first_booking_date']].head())

❌ account_created_date > first_booking_date: 29 Einträge
   (Konto NACH erster Buchung erstellt - unmöglich)

Beispiele:


,user_id,account_created_date,first_booking_date
2,4ft3gnwmtx,2010-09-28,2010-08-02
4,87mebub9p4,2010-09-14,2010-02-18
120,swrvyedlsp,2010-05-30,2010-03-17
357,adq42kzmnv,2010-12-23,2010-04-06
365,176898y1ju,2011-03-28,2010-04-15


## 3. Überprüfung der Konsistenz von first_booking_date und destination_country

In [36]:
# Fehler: keine Buchung, aber destination != NDF
error_booking_destination_inconsistency = df_user[
    ((df_user['first_booking_date'].isna()) & (df_user['destination_country'] != 'NDF')) |
    ((df_user['first_booking_date'].notna()) & (df_user['destination_country'] == 'NDF'))
]
print(f"❌ Kein Buchungsdatum, aber destination_country != 'NDF'\n"
      f"ODER\n"
      f"Buchungsdatum vorhanden, aber destination_country = 'NDF': {len(error_booking_destination_inconsistency)} Einträge")

if len(error_booking_destination_inconsistency) > 0:
    print(f"Beispiele:")
    display(error_booking_destination_inconsistency[['user_id', 'first_booking_date', 'destination_country']].head())
else:
    print("✓ Keine Fehler gefunden")

❌ Kein Buchungsdatum, aber destination_country != 'NDF'
ODER
Buchungsdatum vorhanden, aber destination_country = 'NDF': 0 Einträge
✓ Keine Fehler gefunden


## 4. Analyse der Textspalten (Suche nach Tippfehlern)

In [40]:
# Eindeutige Werte in Textspalten prüfen
string_columns = ['user_gender', 'signup_platform', 'user_language', 
                  'marketing_channel', 'marketing_provider', 'signup_application',
                  'first_device', 'first_web_browser', 'destination_country']

print("Analyse der Textspalten auf Tippfehler:\n")

for col in string_columns:
    unique_vals = df_user[col].value_counts()
    rare_values = unique_vals[unique_vals <= 5]
    
    print(f"{col}: {len(unique_vals)} eindeutige Werte")
    
    if len(rare_values) > 0 and len(unique_vals) > 5:
        print(f"  ⚠️  Seltene Werte (Anzahl ≤ 5): {len(rare_values)}")
        for val, count in rare_values.head().items():
            print(f"      - '{val}': {count}")

Analyse der Textspalten auf Tippfehler:

user_gender: 4 eindeutige Werte
signup_platform: 3 eindeutige Werte
user_language: 25 eindeutige Werte
  ⚠️  Seltene Werte (Anzahl ≤ 5): 3
      - 'ca': 5
      - 'is': 5
      - 'hr': 2
marketing_channel: 8 eindeutige Werte
marketing_provider: 18 eindeutige Werte
  ⚠️  Seltene Werte (Anzahl ≤ 5): 1
      - 'daum': 1
signup_application: 4 eindeutige Werte
first_device: 9 eindeutige Werte
first_web_browser: 52 eindeutige Werte
  ⚠️  Seltene Werte (Anzahl ≤ 5): 22
      - 'Avant Browser': 4
      - 'Opera Mini': 4
      - 'Mozilla': 3
      - 'TheWorld Browser': 2
      - 'Flock': 2
destination_country: 12 eindeutige Werte


## 5. Zusammenfassende Tabelle aller gefundenen Fehler

In [ ]:
# Zusammenfassende Tabelle erstellen
errors_summary = pd.DataFrame({
    'Fehlertyp': [
        'Alter < 18 Jahre',
        'Alter > 90 Jahre',
        'Alter > 80 Jahre (Info)',
        'Alter >= 2000 (Geburtsjahr statt Alter)',
        'first_active_date > account_created_date',
        'account_created_date > first_booking_date',
        'first_active_date > first_booking_date',
        'Keine Buchung, aber destination != NDF',
        'Buchung vorhanden, aber destination = NDF'
    ],
    'Anzahl Einträge': [
        len(young_ages),
        len(old_ages),
        len(age_over_80),
        len(year_ages),
        len(num_errors),
        len(error_booking_destination_inconsistency),
    ]
})

# Nur Zeilen mit Fehlern anzeigen
errors_summary_filtered = errors_summary[errors_summary['Anzahl Einträge'] > 0]

print("=" * 80)
print("ZUSAMMENFASSENDE TABELLE DER GEFUNDENEN FEHLER")
print("=" * 80)
display(errors_summary_filtered)

# Berechne Gesamtzahl (ohne Info-Zeile)
total_errors = errors_summary_filtered[~errors_summary_filtered['Fehlertyp'].str.contains('Info')]['Anzahl Einträge'].sum()
print(f"\nGESAMTZAHL DER EINTRÄGE MIT FEHLERN (mit Überschneidungen): {total_errors}")

## Schlussfolgerungen

In der Datei user.csv wurden folgende Fehlertypen gefunden:

### 1. Altersfehler
- **750 Benutzer** mit Alter >= 2000 (wahrscheinlich Geburtsjahr statt Alter eingegeben)
- **158 Benutzer** mit Alter < 18 Jahre
- **2.701 Benutzer** mit Alter > 90 Jahre (neue Grenze)
- **2.771 Benutzer** mit Alter > 80 Jahre (zur Information)

**Empfehlung:** Unrealistische Alterswerte durch NaN ersetzen

### 2. Datumsreihenfolgefehler
- **0 Einträge** mit first_active_date > account_created_date (durch Datumsvergleich ohne Zeit behoben)
- **29 Einträge** mit account_created_date > first_booking_date
- **0 Einträge** mit first_active_date > first_booking_date (durch Datumsvergleich ohne Zeit behoben)

**Hinweis:** Durch Vergleich nur der Datumsangaben (ohne Zeitstempel) wurden die vorher 
festgestellten Fehler durch die fehlende Zeitkomponente in account_created_date behoben.

**Empfehlung:** Die 29 verbleibenden Einträge in den Quelldaten überprüfen.

### 3. Konsistenz booking_date und destination_country
- **0 Fehler** - Daten sind korrekt ✓

### 4. Potenzielle Tippfehler
- Seltene Werte in user_language, marketing_provider, first_web_browser gefunden
- Die meisten erscheinen legitim (z.B. seltene Browser)